## Google drive와 Google Colab의 연동
-  colab에서 작업 시 google drive에 저장되어 있는 데이터에 접근하기 위해서는 colab을 연동시키는 작업이 필요함.

1. 아래의 코드를 실행.

In [4]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 125036 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.14-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

2. 코드 실행 중 secret key를 입력하라는 빈 칸이 나오면 링크를 클릭하여 구글 아이디로 로그인 하고 key를 발급받아  복사/붙이기로 칸에 입력하고 엔터를 누름(두번 반복).  "Access token retrieved correctly."라는 결과가 나오는 것을 확인.

3. 아래의 코드를 실행하여 "drive" 이름의 폴더를 생성하고 이 폴더를 구글 드라이브와 연동.

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

4. drive 폴더에서 google drive에 있는 파일이 접근되는지 확인.


In [6]:
!cd drive; ls

 영상
'자격증 및 입사 관련 자료'
 잡동사니
'0_[온라인패널조사팀] 인턴지원 - 안수현.doc'
'10월 10일 멘토링쌤 말~.desktop'
 오늘촬영-19947-선.jpg
'제목 없는 프레젠테이션 - 막대 그래프 1.desktop'
'귀농 공동체 사업 - 선 차트 1.desktop'
'귀농 공동체 사업 - 선 차트 1.desktop (7c1c7576)'
'귀농 공동체 사업 - 선 차트 1.desktop (89bf0d5a)'
 2일차
'정보처리기사20190303(해설집).hwp'
'정보처리기사20190303(학생용).hwp'
'정보처리기사20190303(교사용).hwp'
'귀농 공동체 사업 - 막대 그래프 2.desktop'
'시각화 - 막대 그래프 2.desktop'
'고분자화학 입문 3판 솔루션.desktop'
'하둡 설치_하이브(67) 설치.desktop'
'ASC 16-1 - Presentation - Day 02 - 01 Apr - 0800 - Topic 01 - Introduction to Advanced Security Cooperation.desktop'
 cats_and_dogs_small_1.h5
'Colab Notebooks'
'Cover letter Suhyun rev1.docx'
'발병률(천분율).csv'
 DACON.desktop
'DACON 대본.docx'
'DACON 대본.pdf'
'제목 없는 프레젠테이션.desktop'
'제목 없는 스프레드시트.desktop'
'일기 쓰기.desktop'
'통계 자료.desktop'
'금전 내역.desktop'
'귀농 공동체 사업.desktop'
'일정 관리의 사본.desktop'
'하이브 설치 및 오류제거.desktop'
'통계학 정리.desktop'
'기온과 자살률의 연관 분석.desktop'
'오라클 설치하기.desktop'
'자소서 인공지능.desktop'
'관심있는 회사.desktop'
'프로젝트 타임라인.desktop'
 과즙음료_머신러닝모델분석.desktop
 대만여행.desk

- 일정 시간이 지나면 연동이 해제되므로 다시 실행해 주어야 함.

## Tensorboard 사용하기
- 연동된 google drive 상의 폴더에 tensorboard 실행 시의 log 파일을 저장하고 이를 확인하는 예제 

#### Import data

In [7]:
from keras.datasets import mnist   
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

11493376/11490434 [==============================] - 1s 0us/step


In [8]:
train_images.shape

(60000, 28, 28, 1)

#### `LOG_DIR`에 log를 기록할 위치를 지정하고 tensorboard의 실행 port에 접근할 수 있는 링크 생성
- Google drive의 `data/tb_logs` 폴더에 저장하도록 지정함.
- Tensorboard를 실행하고 나면 Google drive에서 해당 폴더가 생성되어 있고 그 안에 log 파일이 있는 것이 확인됨.

In [9]:

LOG_DIR = '/content/drive/data/tb_logs'  

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

import os
if not os.path.exists(LOG_DIR):
  os.makedirs(LOG_DIR)

get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR))

get_ipython().system_raw('./ngrok http 6006 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2019-12-10 08:52:06--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.2.208.1, 52.206.235.84, 3.219.64.173, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.2.208.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  17.7MB/s    in 0.7s    

2019-12-10 08:52:07 (17.7 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


#### Model building 및 fitting 시 TensorBoard 활용
- TensorBoard의 `log_dir`를 위에서 지정한 `LOG_DIR` 안에 실행 시간을 이름으로 하는 폴더로 지정. 
- `model.fit`을 실행한 후 위의 코드 실행 후 출력된 주소를 클릭하면 tensorboard가 실행됨.

In [10]:
## Your answer comes here.
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras import layers
import time 
now = time.strftime("%c")


model = Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks_list = [
    TensorBoard(log_dir=LOG_DIR+'/'+now),
]

model.fit(train_images, train_labels, epochs=5, batch_size=128, validation_split=0.2, callbacks=callbacks_list)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 48000 samples, validate on 12000 samples







Epoch 1/5
48000/48000 [==============================] - 37s 769us/step - loss: 0.2607 - acc: 0.9192 - val_loss: 0.0846 - val_acc: 0.9733

Epoch 2/5
48000/48000 [==============================] - 36s 757us/step - loss: 0.0689 - acc: 0.9784 - val_loss: 0.0587 - val_acc: 0.9829
Epoch 3/5
48000/48000 [==============================] - 36s 755us/step - loss: 0.0462 - acc: 0.9858 - val_loss: 0.0528 - val_acc: 0.9836
Epoch 4/5
48000/48000 [==============================] - 36s 757us/step - loss: 0.0355 - acc: 0.9888 - val_loss: 0.0488 - val_acc: 0.9870
Epoch 5/5
48000/48000 [==============================] - 36s 754us/step - loss: 0.0283 - acc: 0.9915 - val_loss: 0.0404 - val_acc: 0.9882
